<a href="https://colab.research.google.com/github/BaptisteZloch/Weather_forcasting/blob/main/Weather_forcasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Weather forcasting**

Hi I am Baptiste ZLOCH.

This colab notebook is the first part of a project with the goal to predict the sky's state by only capturing climate parameter. 
To capture this parameter we will first use an Arduino Nano 33 BLE sense with only pressure, humidity and temperature sensor using. 

The board : https://store.arduino.cc/arduino-nano-33-ble-sense


Then in the next part of the project we will probably add parameter such as wind speed, direction, sunshine... This will be later.

So first we will extract the humidity, pressure, temperature from our datasets.

This colab use 3 datasets from kaggle to find the best one and to build several model to test accuracy and reach the best. 

We currently use the last model trained on Istanbul dataset.

Feel free to visit my github account to see my others projects.
https://github.com/BaptisteZloch

# Preparing data import with kaggle API

In [1]:
from google.colab import files
files.upload() #upload your JSON token Kaggle API

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"baptistezloch","key":"bbae4093bf46f3d22b36702e36b60c1a"}'}

In [2]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Libraries import

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from keras.utils import to_categorical
from tensorflow.keras.layers.experimental import preprocessing

# Weathercsv dataset

Importing CSV file

In [4]:
!kaggle datasets download -d zaraavagyan/weathercsv
!unzip weathercsv.zip #files extracting
!rm weathercsv.zip

  0% 0.00/10.4k [00:00<?, ?B/s]
100% 10.4k/10.4k [00:00<00:00, 18.2MB/s]
Archive:  weathercsv.zip
  inflating: weather.csv             


In [31]:
df = pd.read_csv('/content/weather.csv',sep=",") #store our dataFrame into Df
df.fillna(0)
df.head()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RISK_MM,RainTomorrow
0,8.0,24.3,0.0,3.4,6.3,NW,30.0,SW,NW,6.0,20,68,29,1019.7,1015.0,7,7,14.4,23.6,No,3.6,Yes
1,14.0,26.9,3.6,4.4,9.7,ENE,39.0,E,W,4.0,17,80,36,1012.4,1008.4,5,3,17.5,25.7,Yes,3.6,Yes
2,13.7,23.4,3.6,5.8,3.3,NW,85.0,N,NNE,6.0,6,82,69,1009.5,1007.2,8,7,15.4,20.2,Yes,39.8,Yes
3,13.3,15.5,39.8,7.2,9.1,NW,54.0,WNW,W,30.0,24,62,56,1005.5,1007.0,2,7,13.5,14.1,Yes,2.8,Yes
4,7.6,16.1,2.8,5.6,10.6,SSE,50.0,SSE,ESE,20.0,28,68,49,1018.3,1018.5,7,7,11.1,15.4,Yes,0.0,No


In [32]:
df.shape

(366, 22)

Data preparing :

In [33]:
df['RainTomorrow'] = pd.Categorical(df['RainTomorrow'])
df['RainTomorrow'] = df.RainTomorrow.cat.codes
df['RainToday'] = pd.Categorical(df['RainToday'])
df['RainToday'] = df.RainToday.cat.codes
df['WindDir3pm'] = pd.Categorical(df['WindDir3pm'])
df['WindDir3pm'] = df.WindDir3pm.cat.codes
df['WindDir9am'] = pd.Categorical(df['WindDir9am'])
df['WindDir9am'] = df.WindDir9am.cat.codes
df.head()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RISK_MM,RainTomorrow
0,8.0,24.3,0.0,3.4,6.3,NW,30.0,12,7,6.0,20,68,29,1019.7,1015.0,7,7,14.4,23.6,0,3.6,1
1,14.0,26.9,3.6,4.4,9.7,ENE,39.0,0,13,4.0,17,80,36,1012.4,1008.4,5,3,17.5,25.7,1,3.6,1
2,13.7,23.4,3.6,5.8,3.3,NW,85.0,3,5,6.0,6,82,69,1009.5,1007.2,8,7,15.4,20.2,1,39.8,1
3,13.3,15.5,39.8,7.2,9.1,NW,54.0,14,13,30.0,24,62,56,1005.5,1007.0,2,7,13.5,14.1,1,2.8,1
4,7.6,16.1,2.8,5.6,10.6,SSE,50.0,10,2,20.0,28,68,49,1018.3,1018.5,7,7,11.1,15.4,1,0.0,0


In [34]:
df_2 = df[["Humidity3pm","Temp3pm","Pressure3pm","RainTomorrow"]]
labels = df_2.pop('RainTomorrow')
features  = np.array(df_2)

## Scikit-Learn : KNN classifier

Creating the model :

In [35]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier()
model.fit(features,labels)
print("Accuracy percentage : {} %".format(100*round(model.score(features,labels),2)))

Accuracy percentage : 87.0 %


Testing the model :

In [37]:
both_y = np.column_stack((labels,model.predict(features)))

TRUE_VALUES = []

for i in range(len(both_y)):
  if both_y[i][0] == both_y[i][1]:
    TRUE_VALUES.append(True)
  else:
    TRUE_VALUES.append(False) 

In [38]:
print("Number of error : {} ".format(np.size(TRUE_VALUES) - np.count_nonzero(TRUE_VALUES)))

Number of error : 46 


# Szeged dataset

## Dataset downloading and data prep

In [ ]:
!kaggle datasets download -d budincsevity/szeged-weather
!unzip szeged-weather.zip
!rm szeged-weather.zip

  0% 0.00/2.23M [00:00<?, ?B/s]
100% 2.23M/2.23M [00:00<00:00, 75.3MB/s]
Archive:  szeged-weather.zip
  inflating: weatherHistory.csv      


In [ ]:
df = pd.read_csv('/content/weatherHistory.csv',sep=",") #store our dataFrame into Df
df = df[["Formatted Date","Summary","Precip Type","Temperature (C)","Humidity","Wind Speed (km/h)","Pressure (millibars)","Daily Summary"]]

df["Humidity"] = df["Humidity"]*100
df["Pressure (millibars)"] = round(df["Pressure (millibars)"])
df["Temperature (C)"] = round(df["Temperature (C)"],2)

import re

df["Formatted Date"] =  [re.sub('[.].*','', str(x)) for x in df["Formatted Date"]]
df["Formatted Date"] = pd.to_datetime(df["Formatted Date"])

for ind in df.index: 
  if df['Precip Type'][ind] == "rain" :
    df['Precip Type'][ind] = 1.0
  elif df['Precip Type'][ind] == "snow":
     df['Precip Type'][ind] = 2.0
  else:
    df['Precip Type'][ind] = 0.0
  if df["Summary"][ind]=='Overcast':
    df["Summary"][ind] = 0
  elif df["Summary"][ind]=='Mostly Cloudy':
    df["Summary"][ind] = 1
  elif df["Summary"][ind]=='Partly Cloudy':
    df["Summary"][ind] = 2
  elif df["Summary"][ind]=='Foggy':
    df["Summary"][ind] = 3
  else:
    df["Summary"][ind] = 4

#nothing = 0, rain = 1, snow = 2

df['Precip Type'] = df['Precip Type'].astype(int)
df['Summary'] = df['Summary'].astype(int)

df = df[["Summary","Temperature (C)","Humidity","Pressure (millibars)"]]

#summary : Partly cloudy, Mostly cloudy, Overcast, Clear, Foggy
df.head()



In [ ]:
df.shape

(96453, 4)

In [ ]:
df_labels = to_categorical(df.pop('Summary'))
df_features = np.array(df)

In [ ]:
from sklearn.model_selection import train_test_split
X, X_test, y, y_test = train_test_split(df_features, df_labels, test_size=0.15)

## Model

### Build the model

In [ ]:
activ_func = 'relu'
model = tf.keras.Sequential([
             tf.keras.layers.Dense(units=3, activation=activ_func, input_shape=(3,)),
             tf.keras.layers.Dense(units=100,activation=activ_func),
             tf.keras.layers.Dense(units=100,activation=activ_func),
             tf.keras.layers.Dense(units=5, activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adagrad(lr=0.001),
             loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

### Train the model

In [ ]:
model.fit(x=X,y=y,epochs=10,use_multiprocessing=True)

### Evaluate the model

In [ ]:
performance=model.evaluate(X_test,y_test, batch_size=32, verbose=1, steps=None, )[1] * 100
print('Model accuracy on test set is ', round(performance), ' %')

453/453 [==============================] - 1s 955us/step - loss: 1.7951 - accuracy: 0.1749
Model accuracy on test set is  17  %


## Convertion

### Converting to TFlite

In [ ]:
# Convert the model to the TensorFlow Lite format without quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model to disk
open("my_model.tflite", "wb").write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmp0uba_q2d/assets


INFO:tensorflow:Assets written to: /tmp/tmp0uba_q2d/assets


46316

### Converting to TFlite micro

In [ ]:
!apt-get -qq install xxd
!echo "const unsigned char model[] = {" > /content/model.h
!cat my_model.tflite | xxd -i      >> /content/model.h
!echo "};"                              >> /content/model.h

import os
model_h_size = os.path.getsize("model.h")
print(f"Header file, model.h, is {model_h_size:,} bytes.")
print("\nOpen the side panel (refresh if needed). Double click model.h to download the file.")

Header file, model.h, is 285,650 bytes.

Open the side panel (refresh if needed). Double click model.h to download the file.


### Converting to TFlite micro (other)

In [ ]:
!pip install tinymlgen
from tinymlgen import port

In [ ]:
c_code = port(model, pretty_print=True)
print(c_code)

INFO:tensorflow:Assets written to: /tmp/tmpuygq6d47/assets


INFO:tensorflow:Assets written to: /tmp/tmpuygq6d47/assets



#ifdef __has_attribute
#define HAVE_ATTRIBUTE(x) __has_attribute(x)
#else
#define HAVE_ATTRIBUTE(x) 0
#endif
#if HAVE_ATTRIBUTE(aligned) || (defined(__GNUC__) && !defined(__clang__))
#define DATA_ALIGN_ATTRIBUTE __attribute__((aligned(4)))
#else
#define DATA_ALIGN_ATTRIBUTE
#endif

const unsigned char model_data[] DATA_ALIGN_ATTRIBUTE = {
	0x20, 0x00, 0x00, 0x00, 0x54, 0x46, 0x4c, 0x33, 0x00, 0x00, 0x00, 0x00, 
	0x00, 0x00, 0x12, 0x00, 0x1c, 0x00, 0x04, 0x00, 0x08, 0x00, 0x0c, 0x00, 
	0x10, 0x00, 0x14, 0x00, 0x00, 0x00, 0x18, 0x00, 0x12, 0x00, 0x00, 0x00, 
	0x03, 0x00, 0x00, 0x00, 0xe8, 0xee, 0x00, 0x00, 0xf4, 0xe0, 0x00, 0x00, 
	0xdc, 0xe0, 0x00, 0x00, 0x3c, 0x00, 0x00, 0x00, 0x04, 0x00, 0x00, 0x00, 
	0x01, 0x00, 0x00, 0x00, 0x0c, 0x00, 0x00, 0x00, 0x08, 0x00, 0x0c, 0x00, 
	0x04, 0x00, 0x08, 0x00, 0x08, 0x00, 0x00, 0x00, 0x08, 0x00, 0x00, 0x00, 
	0x1b, 0x00, 0x00, 0x00, 0x13, 0x00, 0x00, 0x00, 0x6d, 0x69, 0x6e, 0x5f, 
	0x72, 0x75, 0x6e, 0x74, 0x69, 0x6d, 0x65, 0x5f, 0x76, 0x65, 0x72,

# Istanbul dataset

## Dataset downloading and data prep

In [41]:
!kaggle datasets download -d vonline9/weather-istanbul-data-20092019
!unzip weather-istanbul-data-20092019.zip #files extracting
!rm weather-istanbul-data-20092019.zip

  0% 0.00/71.6k [00:00<?, ?B/s]
100% 71.6k/71.6k [00:00<00:00, 63.3MB/s]
Archive:  weather-istanbul-data-20092019.zip
  inflating: Istanbul Weather Data.csv  


In [82]:
df = pd.read_csv('/content/Istanbul Weather Data.csv',sep=",") #store our dataFrame into Df
df = df[["DateTime","Condition","Rain","MaxTemp","MinTemp","AvgWind","AvgHumidity","AvgPressure"]]
df["DateTime"] = pd.to_datetime(df["DateTime"].str.replace(".","/"))

#Now let's associate a code to our str data which are categorical :
#Condition : sunny = 0, Partly cloudy = 1, overcast =2, cloudy=3,  patchy rain possible=4
#Rain : nothing = 0, rain = 1, snow = 2( for another dataset)

#df['Rain'] = pd.Categorical(df['Rain'])
#df['Rain'] = df.Rain.cat.codes
#df['Condition'] = pd.Categorical(df['Condition'])
#df['Condition'] = df.Condition.cat.codes

for ind in df.index: 
  if df['Rain'][ind] != 0.0 :
    df['Rain'][ind] = 1.0
  else:
     df['Rain'][ind] = 0.0
  if df['Condition'][ind]== "Sunny":
    df['Condition'][ind] = 0.0
  elif df['Condition'][ind] == "Partly cloudy" :
    df['Condition'][ind] = 1.0
  elif df['Condition'][ind] == "Overcast" :
    df['Condition'][ind] = 2.0
  elif df['Condition'][ind] == "Cloudy" :
    df['Condition'][ind] = 3.0
  else :
    df['Condition'][ind] = 4.0

df['Rain'] = df['Rain'].astype(int)
df['Condition'] = df['Condition'].astype(int)


df.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to b

,DateTime,Condition,Rain,MaxTemp,MinTemp,AvgWind,AvgHumidity,AvgPressure
0,2019-02-09,1,0,27,22,23,66,1012
1,2019-01-09,1,0,27,22,21,66,1011
2,2019-08-31,4,1,26,22,22,63,1015
3,2019-08-30,1,0,27,22,20,64,1016
4,2019-08-29,1,0,27,23,24,61,1015


In [83]:
df.shape

(3896, 8)

In [84]:

#df = df[["Condition","Rain","MaxTemp","MinTemp","AvgWind","AvgHumidity","AvgPressure"]]
df_features = df.copy()
df_features = df_features[["Condition","MinTemp","AvgHumidity","AvgPressure"]]
df_labels = to_categorical(df_features.pop('Condition'))
df_features = np.array(df_features)

df_features

array([[  22,   66, 1012],
       [  22,   66, 1011],
       [  22,   63, 1015],
       ...,
       [   3,   94, 1021],
       [   1,   89, 1021],
       [   2,   90, 1027]])

In [85]:
df_labels

array([[0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       ...,
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0.]], dtype=float32)

## Model

### Building the model

In [86]:
activ_func = 'relu'
nb_units  = 30
model = tf.keras.Sequential([
             tf.keras.layers.Dense(units=3, activation=activ_func, input_shape=(3,)),
             tf.keras.layers.Dense(units=nb_units,activation=activ_func),
             tf.keras.layers.Dense(units=nb_units,activation=activ_func),
             tf.keras.layers.Dense(units=nb_units,activation=activ_func),
             tf.keras.layers.Dense(units=nb_units,activation=activ_func),
             tf.keras.layers.Dense(units=nb_units,activation=activ_func),
             tf.keras.layers.Dense(units=nb_units,activation=activ_func),
             tf.keras.layers.Dense(units=nb_units,activation=activ_func),
             tf.keras.layers.Dense(units=nb_units,activation=activ_func),
             tf.keras.layers.Dense(units=nb_units,activation=activ_func),
             tf.keras.layers.Dense(units=nb_units,activation=activ_func),
             tf.keras.layers.Dense(units=nb_units,activation=activ_func),
             tf.keras.layers.Dense(units=nb_units,activation=activ_func),
             tf.keras.layers.Dense(units=nb_units,activation=activ_func),
             tf.keras.layers.Dense(units=nb_units,activation=activ_func),
             tf.keras.layers.Dense(units=nb_units,activation=activ_func),
             tf.keras.layers.Dense(units=nb_units,activation=activ_func),
             tf.keras.layers.Dense(units=nb_units,activation=activ_func),
             tf.keras.layers.Dense(units=nb_units,activation=activ_func),
             tf.keras.layers.Dense(units=nb_units,activation=activ_func),
             tf.keras.layers.Dense(units=nb_units,activation=activ_func),
             tf.keras.layers.Dense(units=nb_units,activation=activ_func),
             tf.keras.layers.Dense(units=nb_units,activation=activ_func),
             tf.keras.layers.Dense(units=nb_units,activation=activ_func),
             tf.keras.layers.Dense(units=nb_units,activation=activ_func),
             tf.keras.layers.Dense(units=nb_units,activation=activ_func),
             tf.keras.layers.Dense(units=nb_units,activation=activ_func),
             tf.keras.layers.Dense(units=5, activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.RMSprop(),
             loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),metrics=['acc'])
model.summary()

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_399 (Dense)            (None, 3)                 12        
_________________________________________________________________
dense_400 (Dense)            (None, 30)                120       
_________________________________________________________________
dense_401 (Dense)            (None, 30)                930       
_________________________________________________________________
dense_402 (Dense)            (None, 30)                930       
_________________________________________________________________
dense_403 (Dense)            (None, 30)                930       
_________________________________________________________________
dense_404 (Dense)            (None, 30)                930       
_________________________________________________________________
dense_405 (Dense)            (None, 30)              

### Training the model

In [87]:
model.fit(df_features,df_labels,epochs=100)

Epoch 1/100
122/122 [==============================] - 2s 2ms/step - loss: 1.3197 - acc: 0.4800
Epoch 2/100
122/122 [==============================] - 0s 3ms/step - loss: 1.2097 - acc: 0.5821
Epoch 3/100
122/122 [==============================] - 0s 2ms/step - loss: 1.2435 - acc: 0.5687
Epoch 4/100
122/122 [==============================] - 0s 2ms/step - loss: 1.2220 - acc: 0.5786
Epoch 5/100
122/122 [==============================] - 0s 2ms/step - loss: 1.2053 - acc: 0.5803
Epoch 6/100
122/122 [==============================] - 0s 3ms/step - loss: 1.2262 - acc: 0.5686
Epoch 7/100
122/122 [==============================] - 0s 2ms/step - loss: 1.2376 - acc: 0.5601
Epoch 8/100
122/122 [==============================] - 0s 2ms/step - loss: 1.1994 - acc: 0.5858
Epoch 9/100
122/122 [==============================] - 0s 2ms/step - loss: 1.2180 - acc: 0.5704
Epoch 10/100
122/122 [==============================] - 0s 2ms/step - loss: 1.2166 - acc: 0.5752
Epoch 11/100
122/122 [=================

### Evaluate the model

In [88]:
performance=model.evaluate(df_features,df_labels, batch_size=32, verbose=1, steps=None, )[1] * 100
print('Model accuracy on test set is ', round(performance), ' %')

122/122 [==============================] - 0s 1ms/step - loss: 1.0226 - acc: 0.6204
Model accuracy on test set is  62  %


In [89]:
my_test = np.empty(shape=(3,), dtype=np.int64)
my_test[0] = 22
my_test[1] = 80
my_test[2] = 1025

my_test = my_test.reshape(1, -1)

model.predict(my_test)

array([[0.66057086, 0.20696259, 0.0339927 , 0.05139349, 0.04708035]],
      dtype=float32)

## Model convertion

In [90]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

open("model.tflite", "wb").write(tflite_model)
!apt-get -qq install xxd
!echo "const unsigned char model[] = {" > /content/model.h
!cat model.tflite | xxd -i      >> /content/model.h
!echo "};"                              >> /content/model.h

import os
model_h_size = os.path.getsize("model.h")
print(f"Header file, model.h, is {model_h_size:,} bytes.")

INFO:tensorflow:Assets written to: /tmp/tmpt6iosnfg/assets


INFO:tensorflow:Assets written to: /tmp/tmpt6iosnfg/assets


Header file, model.h, is 656,144 bytes.

Open the side panel (refresh if needed). Double click model.h to download the file.
